<a href="https://colab.research.google.com/github/sujataprasad01/DL-Image-Classification-using-Convolutional-Neural-Network/blob/main/Convolutional_Neural_Network_(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
# import os

# Importing dataset

In [ ]:
kaggle.com/c/dogs-vs-cats/data?select=test1.zip